<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/4_analysis_hate_ar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP final - reconocimiento de discursos discriminatorios en Twitter

## Analisis del Dataset completo marzo 2020 - abril 2021 de X

**Natalia Dedandi**

Se aplica el clasificador seleccionado a todo el conjunto de datos recoelctado para analizar las características del odio en Argentina durante la pandemia

In [1]:
!pip install datasets seaborn


Recupero el conjutno completo de datos: https://huggingface.co/datasets/piuba-bigdata/articles_and_comments



In [2]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("piubamas/articles_and_comments")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tweet_id', 'text', 'title', 'url', 'user', 'body', 'created_at', 'comments'],
        num_rows: 537201
    })
})

In [4]:
ds1 = dataset["train"]

Ejemplo de una fila

In [5]:
ds1[122]

{'tweet_id': '1376979520729849858',
 'text': 'Una fuerte apuesta a la rueda de la agroindustrialización argentina https://t.co/1o6ThCdtxw https://t.co/TLfGhLBjHu',
 'title': 'Una fuerte apuesta a la rueda de la agroindustrialización argentina',
 'url': 'https://www.lanacion.com.ar/economia/campo/una-fuerte-apuesta-a-la-rueda-de-la-agroindustrializacion-argentina-nid30032021/?utm_term=Autofeed&utm_medium=Echobox&utm_source=Twitter#Echobox=1617130198',
 'user': 'LANACION',
 'body': 'Desde el primer día, YPF Agro se propuso proveer al campo argentino de los insumos y las energías necesarias para producir más y mejor, en un contexto donde la demanda de alimentos para la población mundial sigue creciendo.\n\nEl negocio de agro de YPF cerró 2020 con ventas mayores a 1.428 millones de dólares y más de 1,5 millones de toneladas de grano canjeadas, y apuesta a incrementar estos resultados en 2021.\n\nEn ese sentido, lanzó su nueva campaña de comunicación orientada a reforzar la idea de circular

En este conjunto de datos tengo las etiquetas de la clasificación realizada con el modelo BETO:

https://huggingface.co/piuba-bigdata/beto-contextualized-hate-speech

Puede usarse para comparar pero en principio nos interesa recuperar los comentarios completos para aplicarle el clasificador seleccionado

In [6]:
from datetime import datetime, timedelta
from tqdm.auto import tqdm
from collections import Counter

In [13]:
tweets_arg = []

for noticia in tqdm(dataset['train']):

    date_noticia = noticia["created_at"]
    # Convertimos la fecha de la noticia a un objeto de python
    datenoticia = datetime.strptime(date_noticia, "%Y-%m-%dT%H:%M:%S%fZ")
    i=0
    for comment in noticia["comments"]:
        date_comment = comment["created_at"]
        # Convertimos la fecha del comentario a un objeto de python
        datecomment = datetime.strptime(date_comment, "%Y-%m-%dT%H:%M:%S%fZ")
        # anexa comentarios de diarios argentinos
        tweets_arg.append({"id":i,
            "tweet_id_noticia": noticia["tweet_id"],
            "date_tweet": datecomment,
            "text": comment["text"],
            "CALLS":comment["prediction"]["CALLS"],
            "WOMEN":comment["prediction"]["WOMEN"],
            "LGBTI":comment["prediction"]["LGBTI"],
            "RACISM":comment["prediction"]["RACISM"],
            "CLASS":comment["prediction"]["CLASS"],
            "POLITICS":comment["prediction"]["POLITICS"],
            "DISABLED":comment["prediction"]["DISABLED"],
            "APPEARANCE":comment["prediction"]["APPEARANCE"],
            "CRIMINAL":comment["prediction"]["CRIMINAL"]
            })
        i=i+1

  0%|          | 0/537201 [00:00<?, ?it/s]

In [15]:
tweets_arg[11]

{'id': 3,
 'tweet_id_noticia': '1376940852011020288',
 'date_tweet': datetime.datetime(2021, 3, 30, 17, 49, 1, 500000),
 'text': '@LANACION Re largo el sueño',
 'CALLS': 0,
 'WOMEN': 0,
 'LGBTI': 0,
 'RACISM': 0,
 'CLASS': 0,
 'POLITICS': 0,
 'DISABLED': 0,
 'APPEARANCE': 0,
 'CRIMINAL': 0}

In [16]:
# prompt: create a dataframe from tweets_arg

df_tweets_arg = pd.DataFrame(tweets_arg)


In [19]:
df_tweets_arg['HATEFUL']= df_tweets_arg[['CALLS','WOMEN','LGBTI','RACISM','CLASS','POLITICS','DISABLED','APPEARANCE','CRIMINAL']].max(axis=1)

In [20]:
df_tweets_arg.head()

,id,tweet_id_noticia,date_tweet,text,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL,HATEFUL
0,0,1376940813968609288,2021-03-30 17:03:00.900,@clarincom A mi me preocupa el trabajo.. La ev...,0,0,0,0,0,0,0,0,0,0
1,1,1376940813968609288,2021-03-30 17:05:04.500,@clarincom Lo que preocupa. https://t.co/Vmf9V...,0,0,0,0,0,0,0,0,0,0
2,2,1376940813968609288,2021-03-30 17:06:03.100,@clarincom Lo que les preocupa. https://t.co/P...,0,0,0,0,0,0,0,0,0,0
3,3,1376940813968609288,2021-03-30 17:11:02.300,@clarincom Le recomendaríamos al presidente de...,0,0,0,0,0,0,0,0,0,0
4,4,1376940813968609288,2021-03-30 17:26:00.600,@clarincom Para salvar al correo de la quiebra...,0,0,0,0,0,0,0,0,0,0


coloco el mismo orden de las etiquetas que usa el clasificador

In [21]:
# prompt: change the order of columns of df_tweets_arg and put HATEFUL before CALLS

df_tweets_arg = df_tweets_arg[['id', 'tweet_id_noticia', 'date_tweet', 'text', 'HATEFUL', 'CALLS', 'WOMEN',
       'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE',
       'CRIMINAL']]
df_tweets_arg.head()


,id,tweet_id_noticia,date_tweet,text,HATEFUL,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL
0,0,1376940813968609288,2021-03-30 17:03:00.900,@clarincom A mi me preocupa el trabajo.. La ev...,0,0,0,0,0,0,0,0,0,0
1,1,1376940813968609288,2021-03-30 17:05:04.500,@clarincom Lo que preocupa. https://t.co/Vmf9V...,0,0,0,0,0,0,0,0,0,0
2,2,1376940813968609288,2021-03-30 17:06:03.100,@clarincom Lo que les preocupa. https://t.co/P...,0,0,0,0,0,0,0,0,0,0
3,3,1376940813968609288,2021-03-30 17:11:02.300,@clarincom Le recomendaríamos al presidente de...,0,0,0,0,0,0,0,0,0,0
4,4,1376940813968609288,2021-03-30 17:26:00.600,@clarincom Para salvar al correo de la quiebra...,0,0,0,0,0,0,0,0,0,0


In [22]:
# prompt: save df_tweets_arg to csv file

df_tweets_arg.to_csv('tweets_arg.csv', index=False)


In [40]:
# prompt: len of df_tweets_arg

len(df_tweets_arg)


8569648

Lo siguiente lo uso para levantar el csv en lugar de traer los datos (fallo varias veces)

In [28]:
# prompt: open the tweets_arg.csv file and crear a dataframe with that

#import pandas as pd
#df_tweets = pd.read_csv('tweets_arg.csv')


In [41]:
# prompt: create a sample of df_tweets with 10 cases

df_sample = df_tweets_arg.sample(n=5000)


In [42]:
len(df_sample)

5000

In [43]:
# prompt: show the 5 first df_sample

df_sample.head()


,id,tweet_id_noticia,date_tweet,text,HATEFUL,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL
358733,5,1385800665440587784,2021-04-24 06:18:04.000,@latercera Qué importa.\nDe aquí a 3 años Chil...,0,0,0,0,0,0,0,0,0,0
7443670,65,1351946592627077129,2021-01-21 00:35:01.600,@latercera el famoso estallido delictual..se c...,0,0,0,0,0,0,0,0,0,0
2884295,4,1274307361361051648,2020-06-20 11:49:04.900,@clarincom El CM de clarín me lee a mí. https:...,0,0,0,0,0,0,0,0,0,0
5910473,206,1322300355351379969,2020-10-31 04:12:03.400,"@latercera que onda, mientras mas alto el carg...",0,0,0,0,0,0,0,0,0,0
3812167,562,1291036932823285761,2020-08-05 20:29:05.600,@LANACION Este gobierno trata a las mujeres co...,0,0,0,0,0,0,0,0,0,0


Aplico el modelo con finetuning de BETO MULTIETIQUETA

In [44]:
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

checkpoint = "natdebandi/beto_hate_speech_ar_multiclasif"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

tokens = tokenizer(df_sample['text'].tolist(), padding=True, truncation=True, return_tensors="pt")
outputs = model(**tokens)



In [33]:
#outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-6.3324, -8.4825, -7.6645, -7.7967, -7.7177, -8.1828, -7.5653, -7.6700,
         -7.9884, -7.6183],
        [-6.2856, -8.5447, -7.3864, -7.6153, -7.5675, -8.0110, -7.7319, -7.3685,
         -8.1207, -7.8803],
        [-6.2213, -8.8223, -7.2795, -7.5239, -7.7198, -8.0641, -7.7427, -7.6815,
         -8.1344, -7.7841],
        [-6.3335, -8.6191, -7.7441, -7.8408, -7.4901, -8.1081, -7.7943, -7.8330,
         -8.3632, -7.6497],
        [-6.3751, -8.6184, -7.8231, -7.8541, -7.3035, -8.2045, -7.6295, -7.7602,
         -8.3182, -7.8501],
        [-5.9681, -8.3090, -7.7222, -7.7691, -7.7100, -7.0122, -6.9577, -7.0922,
         -8.2451, -7.5750],
        [-6.2157, -8.7861, -7.6355, -7.7987, -7.5957, -8.2380, -7.4715, -7.7786,
         -8.2241, -7.8916],
        [-6.0906, -8.7476, -7.3489, -7.8955, -7.7668, -7.8267, -7.5225, -7.5771,
         -7.7548, -7.8513],
        [-2.3344, -8.1937, -2.4722, -6.5920, -7.9447, -8.2221, -6.6703, -5.9418,
    

In [28]:
logits = outputs.logits
logits.shape

torch.Size([50, 10])

In [29]:
import numpy as np

# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1

In [30]:
model.config.id2label

{0: 'HATEFUL',
 1: 'CALLS',
 2: 'WOMEN',
 3: 'LGBTI',
 4: 'RACISM',
 5: 'CLASS',
 6: 'POLITICS',
 7: 'DISABLED',
 8: 'APPEARANCE',
 9: 'CRIMINAL'}

In [31]:
labels=['HATEFUL','CALLS','WOMEN','LGBTI','RACISM','CLASS','POLITICS','DISABLED','APPEARANCE','CRIMINAL']

In [32]:
predictions

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 

In [33]:
predicted_df = pd.DataFrame(predictions, columns=labels)

In [34]:
predicted_df.head()

,HATEFUL,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# prompt: rename of the columns of predicted_df adding "_pred" to their name

predicted_df = predicted_df.rename(columns=lambda x: x + "_pred")
predicted_df.head()


,HATEFUL_pred,CALLS_pred,WOMEN_pred,LGBTI_pred,RACISM_pred,CLASS_pred,POLITICS_pred,DISABLED_pred,APPEARANCE_pred,CRIMINAL_pred
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
result_df = pd.concat([df_sample, predicted_df], axis=1)

In [39]:
result_df.head()

,id,tweet_id_noticia,date_tweet,text,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,...,HATEFUL_pred,CALLS_pred,WOMEN_pred,LGBTI_pred,RACISM_pred,CLASS_pred,POLITICS_pred,DISABLED_pred,APPEARANCE_pred,CRIMINAL_pred
0,9,1390403146338471937,2021-05-06 20:47:04.600,"@infobae Ahh, un día normal en el conurbano, s...",0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,518,1352294512442437632,2021-01-21 20:57:02.200,"@clarincom El inclusivo es justo con e, no hay...",0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,912,1276684565512323073,2020-06-28 20:29:01.000,@clarincom Más excepciones??,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13,1298585657636421633,2020-08-26 11:53:05.400,@infobae Cómo habrá sentido brancatelli ayer q...,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,153,1346124679342387201,2021-01-04 16:40:01.000,"@LANACION Aclara ""responsabilidad social"", por...",0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
